## Open notebook in:
| Colab                                  Gradient                                                                        
:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Nicolepcx/transformers-the-definitive-guide/blob/master/CH03/ch03_Fine_tune_SAM_with_wandb_optuna.ipynb)                                             

# About This Notebook

This notebook demonstrates a workflow for fine-tuning, training and evaluating a segmentation model using [SAM](https://arxiv.org/abs/2304.02643) (Segment Anything Model) in the context of medical image analysis, specifically for breast cancer datasets.

You first will be installing necessary libraries such as `transformers`, `datasets`, `monai`, and `optuna`. Then you load a breast cancer dataset from Hugging Face's `datasets` library and visualize sample images along with their corresponding ground truth segmentation masks.

Next, you prepare the dataset for training by defining a PyTorch dataset class (`SAMDataset`) that incorporates SAM's required prompts (bounding boxes derived from segmentation masks). You also set up a PyTorch DataLoader to handle batching during training.

The SAM model is loaded from the `transformers` library, and you fine-tune it using a combination of hyperparameter optimization (with [`optuna`](https://optuna.org/)) and logging via [`wandb`](https://wandb.ai/site) to monitor the training process. The training focuses on adjusting the mask decoder parameters of the SAM model, while keeping the vision and prompt encoders frozen.

Finally, the notebook demonstrates inference on unseen data, comparing the predicted segmentation masks to ground truth masks.

Notebook is inspired by: [SAM inference notebook](https://github.com/NielsRogge/Transformers-Tutorials/blob/master/SAM/Run_inference_with_MedSAM_using_HuggingFace_Transformers.ipynb).



# Installs

In [ ]:
!pip install -q datasets==2.21.0 transformers==4.42.4 accelerate==0.32.1 evaluate==0.4.2 monai==1.3.2 optuna==3.6.1 wandb==0.17.7 -qqq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 101.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.1/309.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import optuna
import torch
from PIL import Image
from statistics import mean
from tqdm import tqdm

import monai
import wandb
from datasets import load_dataset
from torch.nn.functional import normalize, threshold
from torch.optim import Adam
from torch.utils.data import DataLoader, Dataset
from transformers import SamModel, SamProcessor




# Load dataset and Show Dataset



Helper function to display a image mask.

In [ ]:
def show_mask(mask, ax, random_color=False):
    """
    Display a mask on a given matplotlib axis.

    Args:
        mask (numpy.ndarray): The mask to be displayed, typically a binary mask.
        ax (matplotlib.axes.Axes): The axis on which to display the mask.
        random_color (bool, optional): If True, the mask will be displayed with a random color.
                                       If False, a default color (light blue) will be used.
                                       Defaults to False.
    """
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

In [ ]:
dataset = load_dataset("nielsr/breast-cancer", split="train")

In [ ]:
dataset

## Display Image

In [ ]:
example = dataset[10]
image = example["image"]


In [ ]:
plt.imshow(image);

# Show Ground Truth Mask

In [ ]:
fig, axes = plt.subplots()
axes.imshow(np.array(image))
ground_truth_seg = np.array(example["label"])
show_mask(ground_truth_seg, axes)
axes.title.set_text(f"Ground truth mask")
axes.axis("off")

# Constructing the PyTorch Dataset

Each dataset entry is composed of:

* Pixel values
* A prompt in the form of a bounding box
* A ground truth segmentation mask

The following function outlines how to generate a bounding box prompt based on the ground truth segmentation mask.

SAM is trained using "prompts," which can take the form of bounding boxes, points, text, or basic masks. The model is then trained to produce the correct mask given the image and the corresponding prompt.


In [ ]:
def get_bounding_box(ground_truth_map):
    """
    Calculate a bounding box around the non-zero regions of a ground truth map with some random perturbation.

    Args:
        ground_truth_map (numpy.ndarray): A 2D array representing the binary ground truth map where
                                          non-zero values indicate the region of interest.

    Returns:
        list: A list representing the bounding box coordinates [x_min, y_min, x_max, y_max].
    """
    y_indices, x_indices = np.where(ground_truth_map > 0)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)

    H, W = ground_truth_map.shape
    x_min = max(0, x_min - np.random.randint(0, 20))
    x_max = min(W, x_max + np.random.randint(0, 20))
    y_min = max(0, y_min - np.random.randint(0, 20))
    y_max = min(H, y_max + np.random.randint(0, 20))
    bbox = [x_min, y_min, x_max, y_max]

    return bbox

In [ ]:
class SAMDataset(Dataset):
    """
    A custom dataset class for processing images and masks using a SAM (Segment Anything Model) processor.

    Args:
        dataset (Dataset): A dataset object containing image data and corresponding labels.
        processor (SamProcessor): A processor object that prepares images and prompts for the SAM model.
    """

    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        """
        Returns the length of the dataset.

        Returns:
            int: The number of items in the dataset.
        """
        return len(self.dataset)

    def __getitem__(self, idx):
        """
        Retrieves an item from the dataset at the specified index and processes it for the SAM model.

        Args:
            idx (int): The index of the item to retrieve.

        Returns:
            dict: A dictionary containing processed inputs and the ground truth mask.
        """
        item = self.dataset[idx]
        image = item["image"]
        ground_truth_mask = np.array(item["label"])

        prompt = get_bounding_box(ground_truth_mask)

        inputs = self.processor(image, input_boxes=[[prompt]], return_tensors="pt")

        inputs = {k: v.squeeze(0) for k, v in inputs.items()}

        inputs["ground_truth_mask"] = ground_truth_mask

        return inputs

In [ ]:
processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

In [ ]:
train_dataset = SAMDataset(dataset=dataset, processor=processor)

In [ ]:
example = train_dataset[0]
for k,v in example.items():
  print(k,v.shape)

In [ ]:
train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [ ]:
batch = next(iter(train_dataloader))
for k,v in batch.items():
  print(k,v.shape)

In [ ]:
batch["ground_truth_mask"].shape

# Load the model

In [ ]:
model = SamModel.from_pretrained("facebook/sam-vit-base")

# Ensure gradients are only computed for the mask decoder
for param_name, parameter in model.named_parameters():
    if param_name.startswith("vision_encoder") or param_name.startswith("prompt_encoder"):
        parameter.requires_grad = False


# Fine-tune the Model

## Login to Weights & Biases

In [ ]:
# Initialize wandb
wandb.init(project='image segmentation')


## Prepare and Run Optuna Hyperparameter Optimization

In [ ]:

device = "cuda" if torch.cuda.is_available() else "cpu"

def objective(trial):
    """
    Objective function for Optuna hyperparameter optimization.

    This function defines the hyperparameter search space, initializes the model,
    optimizer, and loss function, and trains the model for a specified number of epochs.
    It reports the mean loss for each epoch and allows Optuna to prune trials that are
    not promising.

    Args:
        trial (optuna.trial.Trial): A trial object that suggests hyperparameters and
                                    manages the state of the optimization process.

    Returns:
        float: The mean loss over the final epoch of training, which Optuna aims to minimize.
    """
    # Suggested values for the hyperparameters
    lr = trial.suggest_float("lr", 1e-6, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 0, 1e-3)
    num_epochs = trial.suggest_int("num_epochs", 10, 50)
    sigmoid = trial.suggest_categorical("sigmoid", [True, False])
    squared_pred = trial.suggest_categorical("squared_pred", [True, False])

    # Set up the model, optimizer, and loss function
    model.to(device)
    optimizer = Adam(model.mask_decoder.parameters(), lr=lr, weight_decay=weight_decay)
    seg_loss = monai.losses.DiceFocalLoss(sigmoid=sigmoid, squared_pred=squared_pred, reduction='mean')

    model.train()
    for epoch in range(num_epochs):
        epoch_losses = []
        for batch_idx, batch in enumerate(tqdm(train_dataloader)):
            # Forward and backward passes
            outputs = model(pixel_values=batch["pixel_values"].to(device),
                            input_boxes=batch["input_boxes"].to(device),
                            multimask_output=False)
            predicted_masks = outputs.pred_masks.squeeze(1)
            ground_truth_masks = batch["ground_truth_mask"].float().to(device)
            loss = seg_loss(predicted_masks, ground_truth_masks.unsqueeze(1))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_losses.append(loss.item())

        trial.report(np.mean(epoch_losses), epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return np.mean(epoch_losses)

# Create a study object and specify the direction of the optimization
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


In [ ]:
trial.params

In [ ]:
trial.params.get("lr")

In [ ]:
trial.params.get("sigmoid")

# Train the Model on Best Hyperparameter

In [ ]:

# Use Hyperparameter from tuning here
optimizer = Adam(model.mask_decoder.parameters(), lr=trial.params.get("lr"), weight_decay=trial.params.get("weight_decay"))

seg_loss = monai.losses.DiceFocalLoss(sigmoid=trial.params.get("sigmoid"), squared_pred=trial.params.get("squared_pred"), reduction='mean')

In [ ]:
num_epochs = trial.params.get("num_epochs")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.train()

for epoch in range(num_epochs):
    epoch_losses = []
    for batch_idx, batch in enumerate(tqdm(train_dataloader)):
        # Forward pass
        outputs = model(pixel_values=batch["pixel_values"].to(device),
                        input_boxes=batch["input_boxes"].to(device),
                        multimask_output=False)

        # Compute loss
        predicted_masks = outputs.pred_masks.squeeze(1)
        ground_truth_masks = batch["ground_truth_mask"].float().to(device)
        loss = seg_loss(predicted_masks, ground_truth_masks.unsqueeze(1))

        # Backward pass (compute gradients of parameters w.r.t. loss)
        optimizer.zero_grad()
        loss.backward()

        # Optimize
        optimizer.step()
        epoch_losses.append(loss.item())

        # Log sample images and masks to wandb at a specified interval
        if batch_idx % 5 == 0:
            # Select the first sample in the batch for logging
            image_to_log = batch["pixel_values"][0].permute(1, 2, 0).cpu().numpy()  # Convert to HWC format for wandb
            predicted_mask_to_log = predicted_masks[0].cpu().detach().numpy()
            ground_truth_mask_to_log = ground_truth_masks[0].cpu().detach().numpy()

            # Log using wandb
            wandb.log({
                "Input Image": wandb.Image(image_to_log, caption="Input Image"),
                "Predicted Mask": wandb.Image(predicted_mask_to_log, caption="Predicted Mask"),
                "Ground Truth Mask": wandb.Image(ground_truth_mask_to_log, caption="Ground Truth Mask")
            }, commit=False)  # Use commit=False to accumulate logs

    # Log mean loss for the epoch outside the inner loop
    wandb.log({'epoch': epoch, 'mean_loss': mean(epoch_losses)})

    print(f'EPOCH: {epoch}')
    print(f'Mean loss: {mean(epoch_losses)}')




# Inference

A key point to remember: since we utilized the Dice loss with `sigmoid=True`, it's essential to correctly apply a sigmoid activation function to the predicted masks. Therefore, we will not be using the processor's `post_process_masks` method in this case.

In [ ]:
# Take out a random example from the training
idx = 10

# load image
image = dataset[idx]["image"]
image

In [ ]:
# Get box prompt based on ground truth segmentation map
ground_truth_mask = np.array(dataset[idx]["label"])
prompt = get_bounding_box(ground_truth_mask)

# Prepare image + box prompt for the model
inputs = processor(image, input_boxes=[[prompt]], return_tensors="pt").to(device)
for k,v in inputs.items():
  print(k,v.shape)

In [ ]:
model.eval()

with torch.no_grad():
  outputs = model(**inputs, multimask_output=False)

In [ ]:
# Apply sigmoid
medsam_seg_prob = torch.sigmoid(outputs.pred_masks.squeeze(1))
medsam_seg_prob = medsam_seg_prob.cpu().numpy().squeeze()
medsam_seg = (medsam_seg_prob > 0.5).astype(np.uint8)

In [ ]:
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

fig, axes = plt.subplots()

axes.imshow(np.array(image))
show_mask(medsam_seg, axes)
axes.title.set_text(f"Predicted mask")
axes.axis("off")

Compare this to the ground truth segmentation:

In [ ]:
fig, axes = plt.subplots()

axes.imshow(np.array(image))
show_mask(ground_truth_mask, axes)
axes.title.set_text(f"Ground truth mask")
axes.axis("off")